In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from scipy.ndimage import distance_transform_edt as dst
import sys
sys.path.insert(0,r'../src')
from Vsi import Vsi, rock_type, read_image
import os
import tifffile
import glob

In [ ]:
os.chdir(r'../data')
imgs = glob.glob('*.tif')

###### load and visualize data

In [ ]:
fg,ax = plt.subplots(2,2, figsize=(8,8))
ax=ax.ravel()
for i, f_name in enumerate(imgs):
    im = read_image(f_name, pore_phase=1)
    fig_im = ax[i].imshow(im[0], 'gray')
    ax[i].set_title(f_name[:-4])
fg.colorbar(fig_im, ax=ax.tolist())

### Scale-independent Variance

In [ ]:
res={}
for i, f_name in enumerate(imgs):
    im = read_image(f_name, pore_phase=1)
    # ----- get the relative radius-----------
    ds=dst(im[0]) # distance map of a slice, for fast computation. 
    mn_r1 = int(ds.max()) # maximum width of pores is used as minimum radius for moving windows
    mx_r1 = mn_r1+100
    #print('starting radius for '+f_name[:-4]+' is '+str(mn_r1)+' voxels')
    #------run the Vsi--------------
    vf = Vsi(im, no_radii=20,
         no_samples_per_radius=200,
         min_radius=mn_r1, max_radius=mx_r1)
    res[f_name[:-4]] = vf

Plot result

In [ ]:
plt.subplot(211)
for k in res.keys():
    plt.plot(res[k].variance, label=k)
res[k].plot()

plt.subplot(212)
for k in res.keys():
    plt.plot(res[k].variance, label=k)
res[k].plot(fill=0,legend=0)
plt.yscale('log')
plt.tight_layout()

In [ ]:
for k in res.keys():
    print(k,res[k].rock_type())

In [ ]:
# result for 1 sample
res[k].result()

In [ ]:
df = pd.DataFrame(columns=['Radii','Variance','Sample'])
for k in res.keys():
    df0 = res[k].result()
    df0['Sample']=k
    df = pd.concat([df,df0])
df